# Installing and Importing required libraries

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!pip install requests==2.31.0
!pip install fake-useragent
!pip install azure.ai.textanalytics
!pip install azure.core

In [40]:
import urllib.request
import requests
from fake_useragent import UserAgent
import time
import gzip
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.request import Request, urlopen
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import json
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Function / Variable Definitions

In [31]:
# Function to request and retrieve HTML content of a given URL using Selenium
# It sets up a headless Chrome browser, requests the page, waits for it to load, and scrolls to the bottom
# The function returns the HTML of the page
def request_page(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    driver = webdriver.Chrome(options=options)  
    try:
        driver.get(url) #Load the page
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(2, 4))
        html = driver.page_source
        return html
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        driver.quit()

# Function to scrape Amazon reviews from multiple product URLs
# It extracts product ID, name, price, review text, and date, and stores the results in a list
def scrape_amazon_reviews(urls):
    all_reviews_data = [] # List to hold all review data
    
    for url in urls:
        html_content = request_page(url)
        if html_content:
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # Extract Product ID and Name from URL
            try:
                product_id = url.split('/dp/')[1].split('/')[0]
                product_name = url.split('/dp/')[0].split('/')[-1]  
            except IndexError:
                product_id = "ID not found"
                product_name = "Name not found"
            
            price_element = soup.select_one('span.a-price-whole')
            price = price_element.text if price_element else "Price not found"
            review_elements = soup.select('div[data-hook="review"]')
            for review in review_elements:
                content_element = review.select_one('span[data-hook="review-body"] span')
                review_text = content_element.text.strip() if content_element else "Review not found"
                date_element = review.select_one('span[data-hook="review-date"]')
                review_date = date_element.text.strip() if date_element else "Date not found"
                all_reviews_data.append({'Product ID': product_id, 'Product Name': product_name, 'Price': price, 'Review': review_text, 'Date': review_date, 'URL': url}) 
    return all_reviews_data

# List of Amazon product URLs to scrape
# Add more product URLs as needed...
amazon_product_urls = [
    'https://www.amazon.com/Amazon-Essentials-Classic-Fit-Wrinkle-Resistant-Flat-Front/dp/B01JQTGMIO/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Short-Sleeve-Outfit-GALAXY/dp/B0B6DFY3ST/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Boys-Skinny-Jeans-Black/dp/B0CCBC13YQ/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Untucked-Flannel-Button-Blackwatch/dp/B09VS6TS69/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Womens-Short-Sleeve-Uniform/dp/B0B6CX3RKT/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Girls-Sleeve-T-Shirt-Wednesday/dp/B09V8CC9NB/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Girls-Skater-Dress-Floral/dp/B0B6D4B8T4/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Girls-Pull-Joggers-MediumStandard/dp/B09W8Y9LX3/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-282680-womens-Crew-Socks/dp/B09R3MX7LT/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1',
    'https://www.amazon.com/GAP-Vintage-Sleeve-T-Shirt-X-Small/dp/B09VRXCPCW/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1&psc=1'
]

Data has been saved to amazon_product_reviews.csv
Total reviews extracted: 92


# Running Functions & Saving Solutions to CSV file

In [ ]:
all_reviews = scrape_amazon_reviews(amazon_product_urls)

df = pd.DataFrame(all_reviews)
df.to_csv('amazon_product_reviews.csv', index=False)

print("Data has been saved to amazon_product_reviews.csv")
print(f"Total reviews extracted: {len(all_reviews)}")